# Lab 8 — Pipelines CI/CD (SageMaker Pipelines)

In [ ]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.parameters import ParameterString
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.estimator import Estimator
from sagemaker import image_uris
sess=sagemaker.Session(); role=sagemaker.get_execution_role(); region=sess.boto_region_name
bucket=sess.default_bucket(); input_data=ParameterString(name='InputData', default_value=f's3://{bucket}/ml-assoc/l2/output/')
proc=SKLearnProcessor(framework_version='1.2-1', role=role, instance_type='ml.m5.large', instance_count=1)
step_process=ProcessingStep(name='ProcessData', processor=proc, code='/tmp/processing_l2.py')
img=image_uris.retrieve('xgboost', region, '1.7-1')
xgb=Estimator(image_uri=img, role=role, instance_count=1, instance_type='ml.m5.large', output_path=f's3://{bucket}/ml-assoc/l8/model/')
step_train=TrainingStep(name='TrainModel', estimator=xgb, inputs={})
pipeline=Pipeline(name='MLAssocPipeline', parameters=[input_data], steps=[step_process, step_train])
pipeline.upsert(role_arn=role); exe=pipeline.start(parameters={'InputData': input_data.default_value})
print('Pipeline execution:', exe.arn)